### Keras Tuner

In [2]:
# ! pip install keras-tuner

In [3]:
import keras_tuner as kt
from tensorflow import keras

In [4]:
(x_train, y_train), (x_test, y_test) = keras.datasets.fashion_mnist.load_data()

x_train, x_val = x_train[:-10000], x_train[-10000:]
y_train, y_val = y_train[:-10000], y_train[-10000:]

In [5]:
x_train = x_train.reshape((-1, 28, 28, 1)) / 255.0
x_test = x_test.reshape((-1, 28, 28, 1)) / 255.0
x_val = x_val.reshape((-1, 28, 28, 1)) / 255.0

In [6]:
def build_model(hp):
    inputs = keras.Input(shape=(28, 28, 1))

    filters = hp.Int("filters_1", min_value=16, max_value=64, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(inputs)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    filters = hp.Int("filters_2", min_value=32, max_value=128, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    filters = hp.Int("filters_3", min_value=32, max_value=128, step=16)
    x = keras.layers.Conv2D(filters, 3, activation="relu")(x)
    x = keras.layers.MaxPooling2D(pool_size=2)(x)

    x = keras.layers.Flatten()(x)

    outputs = keras.layers.Dense(10, activation="softmax")(x)

    model = keras.Model(inputs, outputs)

    optimizer = hp.Choice("optimizer", values=["rmsprop", "adam"])
    lr = hp.Float("lr", min_value=1e-3, max_value=1e-2, sampling="log")

    if optimizer == "rmsprop":
        opt = keras.optimizers.RMSprop(learning_rate=lr)
    else:
        opt = keras.optimizers.Adam(learning_rate=lr)

    model.compile(opt, loss="sparse_categorical_crossentropy",

                  metrics=["accuracy"])

    return model

In [7]:
model = build_model(kt.HyperParameters())

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 conv2d (Conv2D)             (None, 26, 26, 16)        160       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 32)         0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 3, 3, 32)          9248  

In [8]:
tuner = kt.RandomSearch(build_model, objective="val_accuracy", max_trials=30, seed=0)

tuner.search_space_summary()

Search space summary
Default search space size: 5
filters_1 (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 16, 'sampling': 'linear'}
filters_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
filters_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam'], 'ordered': False}
lr (Float)
{'default': 0.001, 'conditions': [], 'min_value': 0.001, 'max_value': 0.01, 'step': None, 'sampling': 'log'}


In [9]:
#오래 걸림

tuner.search(x_train[:1000], y_train[:1000], batch_size=32, epochs=10, validation_data=(x_val[:1000], y_val[:1000]))

Trial 30 Complete [00h 00m 04s]
val_accuracy: 0.7889999747276306

Best val_accuracy So Far: 0.7889999747276306
Total elapsed time: 00h 02m 31s
INFO:tensorflow:Oracle triggered exit


In [10]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 29 summary
Hyperparameters:
filters_1: 32
filters_2: 128
filters_3: 112
optimizer: adam
lr: 0.004690191543688214
Score: 0.7889999747276306

Trial 21 summary
Hyperparameters:
filters_1: 48
filters_2: 80
filters_3: 48
optimizer: adam
lr: 0.002294874416101842
Score: 0.7839999794960022

Trial 15 summary
Hyperparameters:
filters_1: 48
filters_2: 48
filters_3: 128
optimizer: adam
lr: 0.009317336892175974
Score: 0.781000018119812

Trial 22 summary
Hyperparameters:
filters_1: 48
filters_2: 32
filters_3: 96
optimizer: adam
lr: 0.0042398398536545355
Score: 0.7799999713897705

Trial 02 summary
Hyperparameters:
filters_1: 32
filters_2: 48
filters_3: 80
optimizer: adam
lr: 0.0027907795551781215
Score: 0.7789999842643738

Trial 08 summary
Hyperparameters:
filters_1: 32
filters_2: 48
filters_3: 48
optimizer: adam
lr: 0.0034750681348181137
Score: 0.7789999842643738

Trial 14 summ

In [11]:
best_hps = tuner.get_best_hyperparameters(1)
best_hps[0].values

{'filters_1': 32,
 'filters_2': 128,
 'filters_3': 112,
 'optimizer': 'adam',
 'lr': 0.004690191543688214}

In [12]:
best_models = tuner.get_best_models(1)
final_model = best_models[0]

In [13]:
final_model.evaluate(x_test, y_test)

313/313 [==============================] - 1s 3ms/step - loss: 0.7104 - accuracy: 0.7832


[0.7104488015174866, 0.7832000255584717]